# Prototypage de Modèles - Avec et Sans Feature Engineering

In [3]:
# Importations nécessaires
import pandas as pd
import numpy as np
import yaml
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Importation de nos propres modules
# Ajouter le chemin du dossier 'src' pour que Python puisse trouver nos modules
import sys
sys.path.append('../src')
from features.build_features import preprocess_data

# Pour la gestion du déséquilibre
from imblearn.over_sampling import SMOTE

# --- CHARGEMENT DES DONNÉES ---
with open('../config/main_config.yaml', 'r') as file:
    config = yaml.safe_load(file)
raw_data_path = config['data']['raw_data_path']
df = pd.read_csv(f'../{raw_data_path}')

ModuleNotFoundError: No module named 'src'